In [1]:
from gutenberg.acquire import load_etext
from gutenberg.cleanup import strip_headers

import nltk
from nltk.corpus import wordnet as wn
import inflect

from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
import numpy as np
from collections import Counter, defaultdict

from gensim.utils import tokenize
from itertools import groupby

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.layers import LSTM, RepeatVector
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.optimizers import RMSprop

In [2]:
p = inflect.engine()

In [3]:
nltk.download('wordnet')

pairs = {}
for synset in wn.all_synsets('n'):
    word = synset.name().split('.', 1)[0]
    if not word in pairs:
        pairs[word] = p.plural(word)
len(pairs)

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/mitsuhisa.ohta/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


67176

In [4]:
with open('data/plurals.txt', 'w') as fout:
    for k in sorted(pairs):
        if '_' in k or '-' in k:
            continue
        if k.isdigit():
            continue
        fout.write('%s\t%s\n' % (k, pairs[k]))

In [5]:
p.plural('no')

'noes'

In [6]:
class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one hot integer representation
    + Decode the one hot integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.
        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One hot encode given string C.
        # Arguments
            num_rows: Number of rows in the returned one hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)


In [7]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'


In [8]:
# Parameters for the model and dataset.
INVERT = True

In [9]:
questions = []
expected = []
seen = set()
#with open('data/en_de.txt') as fin:
with open('data/plurals.txt') as fin:
    for line in fin:
        en, de = line.strip().split('\t')
        questions.append(en)
        expected.append(de)

max_question_len = max(len(q) for q in questions)
max_expected_len = max(len(e) for e in expected)
questions = [' ' * (max_question_len - len(q)) + q for q in questions]
expected = [e + ' ' * (max_expected_len - len(e)) for e in expected]
if INVERT:
    questions = [q[::-1] for q in questions]

print('Total addition questions:', len(questions))

Total addition questions: 39929


In [10]:
chars = set(ch for k, v in zip(questions, expected) for ch in k + v)
ctable = CharacterTable(chars)
len(chars)

40

In [11]:
print('Vectorization...')
x = np.zeros((len(questions), max_question_len, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), max_expected_len, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, max_question_len)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, max_expected_len)
print('done')

Vectorization...
done


In [12]:
# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)


Training Data:
(35937, 31, 40)
(35937, 32, 40)
Validation Data:
(3992, 31, 40)
(3992, 32, 40)


In [13]:
# The below is taken from: https://github.com/keras-team/keras/blob/master/examples/addition_rnn.py
RNN = layers.LSTM
HIDDEN_SIZE = 128
LAYERS = 1

print('Build model...')
model = Sequential()
# "Encode" the input sequence using an RNN, producing an output of HIDDEN_SIZE.
# Note: In a situation where your input sequences have a variable length,
# use input_shape=(None, num_feature).
model.add(RNN(HIDDEN_SIZE, input_shape=(max_question_len, len(chars))))
# As the decoder RNN's input, repeatedly provide with the last hidden state of
# RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
# length of output, e.g., when DIGITS=3, max output is 999+999=1998.
#model.add(layers.Dropout(DROP_OUT))
model.add(layers.RepeatVector(max_expected_len))
# The decoder RNN could be multiple layers stacked or a single layer.
for _ in range(LAYERS):
    # By setting return_sequences to True, return not only the last output but
    # all the outputs so far in the form of (num_samples, timesteps,
    # output_dim). This is necessary as TimeDistributed in the below expects
    # the first dimension to be the timesteps.
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))
#    model.add(layers.Dropout(DROP_OUT))

# Apply a dense layer to the every temporal slice of an input. For each of step
# of the output sequence, decide which character should be chosen.
model.add(layers.TimeDistributed(layers.Dense(len(chars))))
model.add(layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Build model...


W0303 23:05:55.412947 139963923275904 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f4b0d41c4e0>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.
W0303 23:05:57.093016 139963923275904 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f4b214df7f0>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
unified_lstm (UnifiedLSTM)   (None, 128)               86528     
_________________________________________________________________
repeat_vector (RepeatVector) (None, 32, 128)           0         
_________________________________________________________________
unified_lstm_1 (UnifiedLSTM) (None, 32, 128)           131584    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 40)            5160      
_________________________________________________________________
activation (Activation)      (None, 32, 40)            0         
Total params: 223,272
Trainable params: 223,272
Non-trainable params: 0
_________________________________________________________________


In [14]:
def create_seq2seq(num_nodes, num_layers):
    num_chars = len(chars)
    question = Input(shape=(max_question_len, num_chars), name='question')
    # repeat = RepeatVector(max_expected_len)(question)
    prev = question
    for _ in range(num_layers):
        lstm = LSTM(num_nodes, return_sequences=True, name='lstm_layer_%d' % (i + 1))(prev)
        prev = lstm
    dense = TimeDistributed(Dense(num_chars, name='dense', activation='softmax'))(prev)
    model = Model(inputs=[question], outputs=[dense])
    optimizer = RMSprop(lr=0.01)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

seq2seq = create_seq2seq(128, 1)

W0303 23:05:57.256747 139963923275904 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f4b0ccccb38>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.


In [15]:

BATCH_SIZE = 2048

# Train the model each generation and show predictions against the validation
# dataset.
for iteration in range(1, 200):
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=10,
              validation_data=(x_val, y_val))
    print()
    print('-' * 50)
    print('Iteration', iteration)
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print(q[::-1] if INVERT else q, '(%s)' % correct, '-', guess)


Train on 35937 samples, validate on 3992 samples
Epoch 1/10
35937/35937 [==============================] - 3s 81us/sample - loss: 2.0254 - accuracy: 0.6684 - val_loss: 1.3531 - val_accuracy: 0.7055
Epoch 2/10
35937/35937 [==============================] - 1s 24us/sample - loss: 1.2275 - accuracy: 0.7063 - val_loss: 1.1028 - val_accuracy: 0.7066
Epoch 3/10
35937/35937 [==============================] - 1s 24us/sample - loss: 1.0541 - accuracy: 0.7161 - val_loss: 1.0166 - val_accuracy: 0.7207
Epoch 4/10
35937/35937 [==============================] - 1s 24us/sample - loss: 0.9745 - accuracy: 0.7214 - val_loss: 0.9050 - val_accuracy: 0.7265
Epoch 5/10
35937/35937 [==============================] - 1s 24us/sample - loss: 0.9176 - accuracy: 0.7249 - val_loss: 0.9067 - val_accuracy: 0.7405
Epoch 6/10
35937/35937 [==============================] - 1s 24us/sample - loss: 0.8775 - accuracy: 0.7328 - val_loss: 0.8556 - val_accuracy: 0.7388
Epoch 7/10
35937/35937 [==============================] -

Epoch 3/10
35937/35937 [==============================] - 1s 24us/sample - loss: 0.7805 - accuracy: 0.7597 - val_loss: 0.7769 - val_accuracy: 0.7641
Epoch 4/10
35937/35937 [==============================] - 1s 24us/sample - loss: 0.7725 - accuracy: 0.7634 - val_loss: 0.7722 - val_accuracy: 0.7623
Epoch 5/10
35937/35937 [==============================] - 1s 24us/sample - loss: 0.7688 - accuracy: 0.7644 - val_loss: 0.7710 - val_accuracy: 0.7650
Epoch 6/10
35937/35937 [==============================] - 1s 24us/sample - loss: 0.7677 - accuracy: 0.7650 - val_loss: 0.7698 - val_accuracy: 0.7637
Epoch 7/10
35937/35937 [==============================] - 1s 24us/sample - loss: 0.7668 - accuracy: 0.7655 - val_loss: 0.7692 - val_accuracy: 0.7660
Epoch 8/10
35937/35937 [==============================] - 1s 24us/sample - loss: 0.7656 - accuracy: 0.7661 - val_loss: 0.7671 - val_accuracy: 0.7654
Epoch 9/10
35937/35937 [==============================] - 1s 23us/sample - loss: 0.7657 - accuracy: 0.7660

35937/35937 [==============================] - 1s 24us/sample - loss: 0.7527 - accuracy: 0.7697 - val_loss: 0.7546 - val_accuracy: 0.7690
Epoch 6/10
35937/35937 [==============================] - 1s 24us/sample - loss: 0.7521 - accuracy: 0.7700 - val_loss: 0.7555 - val_accuracy: 0.7690
Epoch 7/10
35937/35937 [==============================] - 1s 24us/sample - loss: 0.7513 - accuracy: 0.7703 - val_loss: 0.7529 - val_accuracy: 0.7696
Epoch 8/10
35937/35937 [==============================] - 1s 24us/sample - loss: 0.7508 - accuracy: 0.7703 - val_loss: 0.7532 - val_accuracy: 0.7692
Epoch 9/10
35937/35937 [==============================] - 1s 24us/sample - loss: 0.7501 - accuracy: 0.7705 - val_loss: 0.7517 - val_accuracy: 0.7699
Epoch 10/10
35937/35937 [==============================] - 1s 24us/sample - loss: 0.7491 - accuracy: 0.7710 - val_loss: 0.7519 - val_accuracy: 0.7695

--------------------------------------------------
Iteration 7
                          smear (smears             

35937/35937 [==============================] - 1s 23us/sample - loss: 0.8172 - accuracy: 0.7467 - val_loss: 0.7972 - val_accuracy: 0.7503
Epoch 8/10
35937/35937 [==============================] - 1s 24us/sample - loss: 0.7678 - accuracy: 0.7635 - val_loss: 0.7581 - val_accuracy: 0.7691
Epoch 9/10
35937/35937 [==============================] - 1s 23us/sample - loss: 0.7526 - accuracy: 0.7706 - val_loss: 0.7530 - val_accuracy: 0.7708
Epoch 10/10
35937/35937 [==============================] - 1s 23us/sample - loss: 0.7481 - accuracy: 0.7717 - val_loss: 0.7505 - val_accuracy: 0.7697

--------------------------------------------------
Iteration 10
                        hadrian (hadrians                        ) - sariiies                        
                         spread (spreads                         ) - saaiies                         
                      cranberry (cranberries                     ) - carooiiiies                     
                         maniac (maniacs   

35937/35937 [==============================] - 1s 24us/sample - loss: 0.7458 - accuracy: 0.7716 - val_loss: 0.7482 - val_accuracy: 0.7713
Epoch 10/10
35937/35937 [==============================] - 1s 24us/sample - loss: 0.7445 - accuracy: 0.7716 - val_loss: 0.7463 - val_accuracy: 0.7708

--------------------------------------------------
Iteration 13
                      commissar (commissars                      ) - careeiiies                      
                      faceplate (faceplates                      ) - careiiiees                      
                      factorial (factorials                      ) - careeiiies                      
                           boar (boars                           ) - saaes                           
                         loofah (loofahs                         ) - sariies                         
                        rubicon (rubicons                        ) - saraiies                        
                     promptness (pr

...

35937/35937 [==============================] - 1s 24us/sample - loss: 0.0127 - accuracy: 0.9976 - val_loss: 0.0655 - val_accuracy: 0.9840
Epoch 4/10
35937/35937 [==============================] - 1s 25us/sample - loss: 0.0127 - accuracy: 0.9976 - val_loss: 0.0656 - val_accuracy: 0.9842
Epoch 5/10
35937/35937 [==============================] - 1s 24us/sample - loss: 0.0127 - accuracy: 0.9977 - val_loss: 0.0661 - val_accuracy: 0.9841
Epoch 6/10
35937/35937 [==============================] - 1s 25us/sample - loss: 0.0126 - accuracy: 0.9976 - val_loss: 0.0655 - val_accuracy: 0.9842
Epoch 7/10
35937/35937 [==============================] - 1s 25us/sample - loss: 0.0126 - accuracy: 0.9977 - val_loss: 0.0652 - val_accuracy: 0.9842
Epoch 8/10
35937/35937 [==============================] - 1s 24us/sample - loss: 0.0125 - accuracy: 0.9977 - val_loss: 0.0659 - val_accuracy: 0.9841
Epoch 9/10
35937/35937 [==============================] - 1s 24us/sample - loss: 0.0123 - accuracy: 0.9977 - val_loss

35937/35937 [==============================] - 1s 24us/sample - loss: 0.0113 - accuracy: 0.9979 - val_loss: 0.0660 - val_accuracy: 0.9843
Epoch 6/10
35937/35937 [==============================] - 1s 23us/sample - loss: 0.0112 - accuracy: 0.9980 - val_loss: 0.0664 - val_accuracy: 0.9844
Epoch 7/10
35937/35937 [==============================] - 1s 24us/sample - loss: 0.0112 - accuracy: 0.9980 - val_loss: 0.0663 - val_accuracy: 0.9843
Epoch 8/10
35937/35937 [==============================] - 1s 24us/sample - loss: 0.0111 - accuracy: 0.9981 - val_loss: 0.0666 - val_accuracy: 0.9842
Epoch 9/10
35937/35937 [==============================] - 1s 24us/sample - loss: 0.0109 - accuracy: 0.9981 - val_loss: 0.0667 - val_accuracy: 0.9843
Epoch 10/10
35937/35937 [==============================] - 1s 24us/sample - loss: 0.0109 - accuracy: 0.9981 - val_loss: 0.0669 - val_accuracy: 0.9840

--------------------------------------------------
Iteration 188
                         hubbub (hubbubs          

35937/35937 [==============================] - 1s 24us/sample - loss: 0.0154 - accuracy: 0.9969 - val_loss: 0.0677 - val_accuracy: 0.9834
Epoch 8/10
35937/35937 [==============================] - 1s 25us/sample - loss: 0.0152 - accuracy: 0.9969 - val_loss: 0.0673 - val_accuracy: 0.9836
Epoch 9/10
35937/35937 [==============================] - 1s 24us/sample - loss: 0.0148 - accuracy: 0.9970 - val_loss: 0.0674 - val_accuracy: 0.9835
Epoch 10/10
35937/35937 [==============================] - 1s 25us/sample - loss: 0.0145 - accuracy: 0.9971 - val_loss: 0.0671 - val_accuracy: 0.9835

--------------------------------------------------
Iteration 191
                prechlorination (prechlorinations                ) - prechmorniations                
                 overindulgence (overindulgences                 ) - overdidulrences                 
                      pentoxide (pentoxides                      ) - pentoxides                      
                         parvis (parvi    

35937/35937 [==============================] - 1s 24us/sample - loss: 0.0112 - accuracy: 0.9981 - val_loss: 0.0685 - val_accuracy: 0.9838
Epoch 10/10
35937/35937 [==============================] - 1s 25us/sample - loss: 0.0113 - accuracy: 0.9980 - val_loss: 0.0683 - val_accuracy: 0.9841

--------------------------------------------------
Iteration 194
                      adulthood (adulthoods                      ) - adulthoods                      
                    verticality (verticalities                   ) - verticalities                   
                    tentaculata (tentaculatas                    ) - tentaculatas                    
                     christella (christellas                     ) - christellas                     
                      arabesque (arabesques                      ) - arabeqques                      
                     cannonball (cannonballs                     ) - cannnnballs                     
                          assam (a

                          butut (bututs                          ) - bututs                          
                 chrysosplenium (chrysospleniums                 ) - chrysospaediums                 
                    synthesizer (synthesizers                    ) - synthesizers                    
                  assassination (assassinations                  ) - aspassinations                  
                    multiplexer (multiplexers                    ) - multipleeers                    
                      pentoxide (pentoxides                      ) - pentoxides                      
Train on 35937 samples, validate on 3992 samples
Epoch 1/10
35937/35937 [==============================] - 1s 24us/sample - loss: 0.0108 - accuracy: 0.9982 - val_loss: 0.0659 - val_accuracy: 0.9846
Epoch 2/10
35937/35937 [==============================] - 1s 25us/sample - loss: 0.0106 - accuracy: 0.9983 - val_loss: 0.0659 - val_accuracy: 0.9845
Epoch 3/10
35937/35937 [=================

In [16]:
shakespeare = strip_headers(load_etext(100))
tokens = [tuple(word) for word in tokenize(shakespeare, to_lower=True)]
# tokens = [tuple(word) for word in tokenize(plays, to_lower=True)]
token_counts = Counter(tokens)

In [17]:
pairs = [(token[i], token[i + 1], token_id) for token_id, token in enumerate(tokens) for i in range(len(token) - 1)]

In [18]:
pairs[10]

('o', 'r', 2)

In [19]:
help(tokenize)

Help on function tokenize in module gensim.utils:

tokenize(text, lowercase=False, deacc=False, encoding='utf8', errors='strict', to_lower=False, lower=False)
    Iteratively yield tokens as unicode strings, removing accent marks
    and optionally lowercasing the unidoce string by assigning True
    to one of the parameters, lowercase, to_lower, or lower.
    
    Input text may be either unicode or utf8-encoded byte string.
    
    The tokens on output are maximal contiguous sequences of alphabetic
    characters (no digits!).
    
    >>> list(tokenize('Nic nemůže letět rychlostí vyšší, než 300 tisíc kilometrů za sekundu!', deacc = True))
    [u'Nic', u'nemuze', u'letet', u'rychlosti', u'vyssi', u'nez', u'tisic', u'kilometru', u'za', u'sekundu']



In [20]:
tokens[5]

('s', 'h', 'a', 'k', 'e', 's', 'p', 'e', 'a', 'r', 'e')